# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
soup = soup(html, 'html.parser')

In [ ]:
# Extract all rows of data
rows = soup.find_all('table', class_='table')
rows

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list of headers
headers_list = []

# Loop through the scraped data to create a list of columns
for row in rows:
    headers = row.find_all('th')
    for header in headers:
        title = header.text
        headers_list.append(title)
        
headers_list

In [ ]:
# Create an empty list of row data
mars_weather = []

# Loop through the scrapped data to create a list of row data
for row in rows:
    data = row.find_all('td')
    for i in data:
        mars_weather.append(i.text)
mars_weather

In [ ]:
# Split the elements in the row data list after every 7 values to match the columns
n=7
mars_data = [(mars_weather[i:i + n]) for i in range(0, len(mars_weather), n)]
mars_data

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
mars_df = pd.DataFrame(mars_data, columns=headers_list)

In [ ]:
# Confirm DataFrame was created successfully
mars_df

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_df.dtypes

In [ ]:
# Change data types for data analysis
mars_df['terrestrial_date'] = pd.to_datetime(mars_df['terrestrial_date'])
mars_df[['sol','ls','month']] = mars_df[['sol','ls','month']].astype(int)
mars_df[['min_temp','pressure']] = mars_df[['min_temp','pressure']].astype(float)

In [ ]:
# Confirm type changes were successful by examining data types again
mars_df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
mars_df['month'].max()

In [ ]:
# 2. How many Martian days' worth of data are there?
mars_df['sol'].count()

In [ ]:
# 3. What is the average low temperature by month?
avg_temp = mars_df.groupby(['month']).mean()['min_temp']
avg_temp

In [ ]:
# Plot the average temperature by month
avg_temp.plot.bar(xlabel='Month', 
                  ylabel='Temperature in Celsius', 
                  title='Average Daily Minimum Temperature on Mars by Month')
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
avg_temp = avg_temp.to_frame()
temp_sorted = avg_temp.sort_values('min_temp')
temp_sorted.plot.bar(xlabel='Month', 
                  ylabel='Temperature in Celsius',
                  title='Average Daily Minimum Temperature on Mars by Month',
                    legend=False)
plt.show()

In [ ]:
# Average pressure by Martian month
avg_pressure = mars_df.groupby(['month']).mean()['pressure']
avg_pressure

In [ ]:
# Plot the average pressure by month
avg_pressure = avg_pressure.to_frame()
pressure_sorted = avg_pressure.sort_values('pressure')
pressure_sorted.plot.bar(xlabel='Month', 
                  ylabel='Atmospheric Pressure', 
                  title='Average Daily Pressure on Mars by Month',
                    legend=False)
plt.show()

In [ ]:
# How many terrestrial (earth) days are there in a Martian year?
plt.plot(mars_df['sol'], mars_df['min_temp'])
plt.xlabel('Number of Martian days')
plt.ylabel('Temperature in Celsius')
plt.title('Daily Minimum Temperatures on Mars')
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_df = mars_df.set_index('id')
mars_df.to_csv('mars_weather.csv',  encoding='utf8')

In [ ]:
browser.quit()